ابتدا کتاب خانه های مورد نظر برای اندازه گیری زمان مصرفی و فضای اشغال شده را ایمپورت می‌کنیم

In [1]:
import time
import tracemalloc

سپس کلاسی میسازیم تا اطلاعات ورودی که داده می‌شود را گرفته و اطلاعات مورد نیاز مانند محاسبه هیوریستیک یا سایر موارد را انجام دهد

In [32]:
class State:

    def __init__(self, state_list):
        self.state = state_list
        self.children, self.children_moves = self.available_moves()
        self.parent = None
        self.gn = 0
        self.hn = 0
        self.gn = 0
        self.depth = 0
    
    def get_depth(self):
        self.depth = self.parent.depth + 1
            
    def calculate_gn(self):
        self.gn = self.parent.gn + 1

    def calculate_fn(self):
        self.fn = self.gn + self.hn

    def calculate_hn_misplaced(self, state):
        #using number of misplaced tiles
        result = 0
        for i in range(len(self.state)):
            if self.state[i] != i+1:
                result += 1
        if self.state[8] == 0:
            result -= 1
        self.hn = result
    
    def calculate_hn(self, state):
        #using manhattan distance
        dis = 0
        for i in range(len(self.state)):
            dis += abs(i//3 - (self.state[i]-1)//3) + abs(i%3 - (self.state[i]-1)%3)
        self.hn = dis

    def available_moves(self):
        # find available moves (children) of a state
        result = list()
        children_moves = {}
        index_of_0 = self.state.index(0)
        up = [0,1,2]
        down = [6,7,8]
        right = [2,5,8]
        left = [0,3,6]
        x = self.state.copy()
        if index_of_0 not in up:
            x[index_of_0-3], x[index_of_0] = self.state[index_of_0], self.state[index_of_0-3]
            result.append(x)
            children_moves[str(x)] = "↑"
        x = self.state.copy()
        if index_of_0 not in down:
            x[index_of_0+3], x[index_of_0] = self.state[index_of_0], self.state[index_of_0+3]
            result.append(x)
            children_moves[str(x)] = "↓"
        x = self.state.copy()
        if index_of_0 not in left:
            x[index_of_0-1], x[index_of_0] = self.state[index_of_0], self.state[index_of_0-1]
            result.append(x)
            children_moves[str(x)] = "←"
        x = self.state.copy()
        if index_of_0 not in right:
            x[index_of_0+1], x[index_of_0] = self.state[index_of_0], self.state[index_of_0+1]
            result.append(x)
            children_moves[str(x)] = "→"
        return result, children_moves

    def __eq__(self, other):
        return self.state == other.state
    
    def __repr__(self) -> str:
        return str(self.state)
goal = State([1,2,3,4,5,6,7,8,0])

در این تابع به ازای تمامی مثال های قرار گرفته در فایل سوال به اجرای الگوریتم و دادن خروجی می‌پردازد

In [3]:
def print_output(func):
    # Only should be used with A*, BFS, DFS, UCS (NOT IDS)
    with open("Assignment 2 - Examples.txt") as file:
        examples = file.read()
    for i in range(len(examples)):
        list1=[]
        if examples[i] == "{":
            i += 1
            while examples[i] != "}":
                if examples[i] != ',' and examples[i] != ' ':
                    list1.append(int(examples[i]))
                i += 1
            find_route(func(State(list1),0), State(list1))
            print('\n')


این تابع با توجه به نود های بررسی شده تا انتهای مسیر دوباره به عقب برگشته و انتخابی که نود والد آن نود انجام داده تا به نود فرزند برسد را ذخیره کرده تا در نهایت مسیر درست از ابتدا تا رسیدن به استیت هدف را به دست بی‌آوریم

In [4]:
def find_route(visited, start_state):
    route = {}
    path_state = visited[-1]
    while path_state != start_state:
        route[str(path_state)] = path_state.parent.children_moves[str(path_state)]
        path_state = path_state.parent
    print(" Steps =", len(route.values()),"\n",start_state,', '.join(list(route.values())[::-1]))

در تابع بعد با توجه به مقادیری که برای f(n) تولید می‌شود اندازه آن ها به دست می‌آید و با ترتیب وارد صف می‌شوند تا در الگوریتم A* استفاده شوند
سپس الگوریتم A* نوشته شده که در آن در ابتدا و انتها زمان و مموری را اندازه گرفته و در اخر آن را خروجی می‌دهیم

In [5]:
def append_in_order(queue,state):
    # To append in the list according to fn
    result = 0
    for s in queue:
        if s.fn > state.fn:
            break
        result +=1
    queue.insert(result, state)
    return queue

def A_Star(state, checker):
    exceeding_time = 120
    start = time.perf_counter()
    start_space = tracemalloc.start()
    queue = [state]
    visited = []
    while queue:
        time_passed = time.perf_counter()-start > exceeding_time
        if time_passed:
            print("Time limit exceeded!\n")
            checker = 1
            break
        expanded = queue.pop(0)
        if expanded == goal:
            visited.append(expanded)
            break
        for x in expanded.children:
            child = State(x)
            child.parent = expanded
            child.calculate_gn()
            child.calculate_hn(goal)
            child.calculate_fn()
            if child not in visited:
                append_in_order(queue, child)
                visited.append(child)
    memory = tracemalloc.get_traced_memory()[1]
    end_space = tracemalloc.stop()
    end = time.perf_counter()
    if checker == 0:
        print(' Time:',(end-start),"secs")
        print(' Memory:',memory,"bytes")
    return visited

پیاده سازی الگوریتم BFS
در الگوریتم ها ورودی چکر تنها برای بررسی چاپ یا عدم چاپ خروجی است.

In [33]:
def BFS(state, checker):
    exceeding_time = 120
    start = time.perf_counter()
    start_space = tracemalloc.start()
    queue = [state]
    visited = []
    while queue:
        time_passed = time.perf_counter()-start > exceeding_time
        if time_passed:
            print("Time limit exceeded!\n")
            checker = 1
            break
        expanded = queue.pop(0)
        if expanded == goal:
            visited.append(expanded)
            break
        for x in expanded.children:
            child = State(x)
            child.parent = expanded
            if child not in visited:
                visited.append(child)
                queue.append(child)
    memory = tracemalloc.get_traced_memory()[1]
    end_space = tracemalloc.stop()
    end = time.perf_counter()
    if checker == 0:
        print(' Time:',(end-start),"secs")
        print(' Memory:',memory,"bytes")
    return visited

پیاده سازی الگوریتم DFS

In [34]:
def DFS(state, checker, max_depth=20):
    exceeding_time = 120
    start = time.perf_counter()
    start_space = tracemalloc.start()
    queue = [state]
    visited = []
    while queue:
        time_passed = time.perf_counter()-start > exceeding_time
        if time_passed:
            print("Time limit exceeded!\n")
            checker = 1
            break
        expanded = queue.pop()
        if expanded == goal:
            visited.append(expanded)
            break
        if expanded.depth < max_depth:
            for x in expanded.children:
                child = State(x)
                child.parent = expanded
                child.get_depth()
                if child not in visited:
                    visited.append(child)
                    queue.insert(0,child)
    memory = tracemalloc.get_traced_memory()[1]
    end_space = tracemalloc.stop()
    end = time.perf_counter()
    if checker == 0:
        print(' Time:',(end-start),"secs")
        print(' Memory:',memory,"bytes")
    return visited

پیاده سازی الگوریتم IDS

In [35]:
def IDS(state):
    start = time.perf_counter()
    start_space = tracemalloc.start()
    for depth in range(1, 30):
        print(" Depth: ",depth)
        result = DFS(state,1,depth)
        if result[-1] == goal:
            break
    memory = tracemalloc.get_traced_memory()[1]
    end_space = tracemalloc.stop()
    end = time.perf_counter()
    print(' Time:',(end-start),"secs")
    print(' Memory:',memory,"bytes")
    return result

پیاده سازی الگوریتم UCS

In [36]:
def UCS(state, checker):
    # same as A* without h(n)
    exceeding_time = 120
    start = time.perf_counter()
    start_space = tracemalloc.start()
    queue = [state]
    visited = []
    while queue:
        time_passed = time.perf_counter()-start > exceeding_time
        if time_passed:
            print("Time limit exceeded!\n")
            checker = 1
            break
        expanded = queue.pop(0)
        if expanded == goal:
            visited.append(expanded)
            break
        for x in expanded.children:
            child = State(x)
            child.parent = expanded
            child.calculate_gn()
            child.calculate_fn()
            if child not in visited:
                append_in_order(queue, child)
                visited.append(child)
    memory = tracemalloc.get_traced_memory()[1]
    end_space = tracemalloc.stop()
    end = time.perf_counter()
    if checker == 0:
        print(' Time:',(end-start),"secs")
        print(' Memory:',memory,"bytes")
    return visited

در انتها نحوه ورودی دادن به الگوریتم ها با توجه به حالات مختلف آورده شده است

In [31]:
# Data produced after "time limit exceeded!" message is garbage.
entry = State([0,1,2,4,5,3,7,8,6]) # to give one input at a time

#find_route(IDS(entry), entry) # form of giving input to IDS (one input at a time)

find_route(UCS(entry, 0), entry) #form of giving input to algorithms except IDS (one input at a time)


#print_output(BFS) # for all algorithms except IDS (all inputs together)

 Time: 0.014901900000040769 secs
 Memory: 4735927 bytes
 Steps = 4 
 [0, 1, 2, 4, 5, 3, 7, 8, 6] →, →, ↓, ↓
